In [ ]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib
!pip install mpl_finance
from google.colab import drive
drive.mount('/content/drive')

(Reading database ... 144637 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras import *
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    # tf.config.experimental.set_virtual_device_configuration(gpu0,
    # [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import talib
import pandas_datareader as pdr
import mpl_finance as mpf
import seaborn as sns
import datetime as datetime
from sklearn import preprocessing
import requests
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from PIL import Image, ImageDraw, ImageFont
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import csv
import statistics
import math
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SpatialDropout2D, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D,MaxPooling1D
from tensorflow.keras.layers import Activation,Dense ,Dropout ,GRU, ConvLSTM2D ,LSTM ,Bidirectional,TimeDistributed,Flatten, Conv1D, MaxPooling1D,BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers 
from tensorflow.keras.layers import LayerNormalization
from tensorflow.python.keras.layers import Input, Embedding, Dot, Reshape, Dense
from tensorflow.python.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adamax, Adam
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
def exponential_smoothing(days, alpha, numdata):
  # assign weights
  weights = []
  i = days 
  while i>0:
    weights.append( math.exp(-alpha*i) )
    i-=1
  weights = np.array(weights) / sum(weights)
  # return data after smoothing
  results = []
  results += numdata[:days]
  for i in range(days, len(numdata)):
    results.append((np.dot( np.array(weights), np.array(numdata[i-days:i]))))
  return results

classes =['0', '1']
def to_onelist(text):
  onehot = [0] * 2
  onehot[ int(classes.index(str(text)) ) ] = 1
  label_list = np.array(onehot)
  return label_list

In [ ]:
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002",
            "2105", "2207", 
            # "2227", 
            "2301", "2303", "2308", "2317", "2327",
            "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610",
            "2633", "2801", "2823", "2880", "2881", "2882", "2883", "2884", "2885",
            "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711",
            "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

In [ ]:
i=0
while i<53 :
  print(stockNum[i: i+10])
  i+=10
user_choice = []
i= 0 
while i <10:
  choice = str((input("請輸入第"+str(i)+"支股票")))
  if choice not in stockNum:
    print("欲搜尋的股票不在清單上 :)")
    continue
  if choice in user_choice:
    print("error 重複輸入")
    continue
  user_choice.append(choice)
  i+=1
print("所輸入股票為", user_choice)
stockNum = user_choice
print("開始載入過去資料進行訓練")

['1101', '1102', '1216', '1301', '1326', '1402', '2002', '2105', '2207', '2301']
['2303', '2308', '2317', '2327', '2330', '2352', '2357', '2382', '2395', '2408']
['2412', '2454', '2474', '2609', '2610', '2633', '2801', '2823', '2880', '2881']
['2882', '2883', '2884', '2885', '2886', '2887', '2888', '2890', '2891', '2892']
['2912', '3008', '3045', '3711', '4904', '4938', '5871', '5880', '6505', '9904']
['9910']
請輸入第0支股票1101
請輸入第1支股票1102
請輸入第2支股票
欲搜尋的股票不在清單上 :)
請輸入第2支股票1216
請輸入第3支股票1301
請輸入第4支股票3008
請輸入第5支股票3045
請輸入第6支股票3711
請輸入第7支股票4938
請輸入第8支股票2330
請輸入第9支股票5880
所輸入股票為 ['1101', '1102', '1216', '1301', '3008', '3045', '3711', '4938', '2330', '5880']
開始載入過去資料進行訓練


In [ ]:
#全部時間
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
start = '2005/01/01'
end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [ ]:
pd.options.mode.chained_assignment = None 

train_x = []
test_x = []
val_x = []
val_y = []
train_y = []
test_y = []

for stockno in stockNum:
    x = []
    if int(stockno) == 2227: continue
    df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5/'+str(stockno)+'.csv')
    record = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5/'+str(stockno)+'mean&stdev.csv')
    df.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
    df = df.drop(['Date'], axis=1)
    df = df.drop(['MarginPurchaseLimit'], axis=1)
    df = df.drop(['ShortSaleLimit'], axis=1)
    df = df.dropna(axis=0, how='any')
    df.reset_index(drop=True, inplace=True)
    days_before = 30
    for i in range(days_before, len(df)):
        tmp = df[i-days_before:i]
        for col in df.columns:
          if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
          mean = record[str(col)+'_mean'][i-30]
          stdev = record[str(col)+'_stdev'][i-30]
          def test(s): 
            return (float(s)-mean)/stdev
          tmp[str(col)].map(test)
        x.append( np.array(np.array(tmp).tolist()[0: days_before]).reshape(days_before,len(tmp.columns)))
    y = list(df['y_label'][days_before:]) 
    #train:val:test = 8:1:1
    tr_va = int(len(x)*8/10)
    va_te = int(len(x)*9/10)
    # print("no: ", tr_va, "no: ", va_te, "leng: ", len(y))
    train_x = train_x + x[:tr_va]
    train_y = train_y + y[:tr_va]
    val_x = val_x + x[tr_va: va_te]
    val_y = val_y + y[tr_va: va_te]
    test_x = test_x + x[va_te:]
    test_y = test_y +y[va_te:]
    print(str(stockno)+" loaded....")
    # print(len(train_x), len(val_x), len(test_x))
print("資料載入完成")

1101 loaded....
1102 loaded....
1216 loaded....
1301 loaded....
3008 loaded....
3045 loaded....
3711 loaded....
4938 loaded....
2330 loaded....
5880 loaded....
資料載入完成


In [ ]:
y = [] 
print(train_y)
for i in range(len(train_y)):
  y.append(to_onelist(str(train_y[i])))
train_y = y

y = [] 
for i in range(len(test_y)):
  y.append(to_onelist(str(test_y[i])))
test_y = y

y = [] 
for i in range(len(val_y)):
  y.append(to_onelist(str(val_y[i])))
val_y = y
train_x = np.array(train_x).reshape(len(train_x), 30, 23)
test_x  = np.array(test_x).reshape(len(test_x), 30, 23)
val_x   = np.array(val_x).reshape(len(val_x), 30, 23)
train_y = np.array(train_y).reshape(len(train_y),2)
test_y  = np.array(test_y).reshape(len(test_y),2)
val_y   = np.array(val_y).reshape(len(val_y),2)
def lstm4_model(input_shape , n_classes):
  model = Sequential()
  model.add(BatchNormalization(input_shape=(input_shape[0],input_shape[1])))
  model.add(LSTM(25,return_sequences=True))
  model.add(BatchNormalization())
  model.add(LSTM(12,return_sequences=True))
  model.add(BatchNormalization())
  model.add(LSTM(6,return_sequences=True))
  model.add(BatchNormalization())
  model.add(LSTM(3,return_sequences=True))
  model.add(Flatten())
  model.add(BatchNormalization())
  model.add(Dense(90, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
  model.add(BatchNormalization())
  model.add(Dense(25, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
  model.add(BatchNormalization())
  model.add(Dense(2, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(l=0.001)))
  lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=50, decay_rate=0.9)
  model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])
  model.summary()
  return model
model = lstm4_model((30, 23), 2)
callback = EarlyStopping(monitor="val_loss", patience=25, verbose=1, mode="auto")
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(train_x, train_y , epochs=100, batch_size= len(train_x), validation_data=(val_x, val_y), callbacks=[callback, tensorboard_callback] )
print("model 已訓練結束")
prediction_y = []
truth_y = []
for i in range(len(test_y)):
  prediction_y.append(int(classes[np.argmax(model.predict(test_x[i].reshape(1,30,23)))]))
  truth_y.append(int(classes[np.argmax(test_y[i])]))
print(" model 整體 Accuracy: ")
print( 1 - (np.sum(np.logical_xor(np.array(prediction_y), np.array(truth_y))))/len(test_y) )
model.save('/content/drive/My Drive/Program_Trading_Code/temporary.h5')

[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
#選定欲回測的model
from keras.models import load_model
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
model = load_model('/content/drive/My Drive/Program_Trading_Code/temporary.h5')

#設定 股票資料
#validaiton 日期區間
start = '2018/08/01'
end = '2019/06/03'

s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [ ]:
wala_wala = 0
wala_wala_our = 0
wala_wala_origin = 0
#asset_at_the_start = 5.1 * 10**7
DIC_ROR = {}
for stock_num in stockNum:
  #先算出之後回測用的價位
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  #融資表格
  stock_time = str(Price_df.index[0]).split(" ")[0]
  # print(stock_time)
  #load in 融資融券
  url = "https://api.finmindtrade.com/api/v3/data"
  parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
  data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
  data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
  data = data.set_index('date')
  # 合併這兩個表格
  new_df = Price_df.join(data)
  new_df = new_df.drop(['stock_id'], axis=1)
  new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
  # 處理三大法人投資情況 視情況加入
  url = "https://api.finmindtrade.com/api/v3/data"
  parameter = {
    "dataset": "InstitutionalInvestorsBuySell",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  df = pd.DataFrame()
  z = 0 
  i = 0
  while i < len(data)  :
    a = data['date'][i]
    count = 0
    sell = 0
    buy = 0 
    if i+6 < len(data)-1:
        for j  in range(i, i+6):
            if data['date'][j]== a :
              count += 1
              sell += data['sell'][j]
              buy += data['buy'][j]
    else:
      for j in range(i, len(data)):
        if data['date'][j] == a :
          count += 1
          sell += data['sell'][j]
          buy += data['buy'][j]
    df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    i+=count
  new_df = new_df.join(df.set_index('date'))
  new_df = new_df.fillna(axis=0,method='ffill')
  new_df = new_df.drop(['Note'], axis=1)
  # gpy = data.groupby('date').sum()
  # gpy.rename(columns={'buy':'buy_volume', 'sell': 'sell_volume'}, inplace=True)
  # gpy['bs_ratio'] = gpy['buy_volume'] / (gpy['sell_volume']+1)
  # df = gpy
  # new_df = new_df.join(df)
  # new_df = new_df.dropna(axis=0, how='any')
  # new_df = new_df.drop(['Note'], axis=1)
  new_df.reset_index(drop=True, inplace=True)
  for columnName in new_df.columns:
    if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    # print(columnName)
    new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    days = 10
    ans = [0]*days
    for i in range(days,len(new_df)):
      Pi = new_df['Close'][i]
      Pj = new_df['Close'][i-days]
      if Pi > Pj : y = 1
      else: y = 0
      ans.append(y)
  new_df['y_label']= ans
  #更改index 將日期更改為
  new_df = new_df.reset_index()
  new_df.rename(columns={'index':'Date'}, inplace=True)
  time_list = list(new_df['Date'])
  new_df = new_df.drop(['Date'], axis=1)
  new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
  new_df = new_df.drop(['ShortSaleLimit'], axis=1)
  # new_df = new_df.drop(['level_0'], axis=1)
  Price_df = Price_df.reset_index()

  # start evaluaion 
  ###############一般回測################################
  left_money = 10**6
  own_asset = 10**6
  time_slide = 30 
  own_stock = 0 
  money_record = [left_money]*time_slide
  origin_money = 10**6 #紀錄開場進最後出
  origin_hold_stock = 0 
  for i in range(time_slide, len(new_df)): 
    tmp = new_df[i-time_slide:i]
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model.predict(x.reshape(1,30,23)))])
    price = int(Price_df['Close'][i]*100)/100
    # if i == time_slide: #起始買
    #   origin_hold_stock += int( left_money / ( price * (1 + 0.001425)) ) # 包含手續費可以購買的股數
    #   origin_money = origin_money - origin_hold_stock*price*(1 + 0.001425)
    # elif i == len(new_df)-1: #結束賣
    #   origin_money = origin_money + origin_hold_stock*price*(1 - 0.001425 - 0.003)
    if predict == 1:
      if own_stock ==0:
        # print("買入")
        own_stock += int( left_money / ( price * (1 + 0.001425)) )
        left_money = left_money - own_stock*price*(1 + 0.001425)
        # print("目前價格： " , price, "日期: ",time_list[i])
        # print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
    elif predict == 0 :
      if own_stock > 0:
        # print("賣出")
        left_money += own_stock*price*(1 - 0.001425 - 0.003)
        own_stock = 0
        # print("目前價格： " , price, "日期: ", time_list[i])
        # print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
    money_record.append(left_money+own_stock*price)
  print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(10**6))*100/(10**6),"%")
  #print(str(stock_num)+"最後資產 ：", int(origin_money),"報酬率：",(int(origin_money)-(10**6))*100/(10**6),"%（開場進 最後出）")
  #wala_wala_weight += asset_at_the_start * DIC[stock_num]
  wala_wala += 10**6
  wala_wala_our += int(money_record[-1])
  wala_wala_origin += int(origin_money)
  DIC_ROR[str(stock_num)] = (int(money_record[-1])-(10**6))*100/(10**6)
  print(DIC_ROR[str(stock_num)])
print("測試結束")
print("our總投資報酬率：", (wala_wala_our - wala_wala)*100 / wala_wala, "%")
# print("origin總投資報酬率：", (wala_wala_origin - wala_wala)*100 / wala_wala, "%")

1101最後資產 ： 726554 報酬率： -27.3446 %
-27.3446
1102最後資產 ： 1418553 報酬率： 41.8553 %
41.8553
1216最後資產 ： 720510 報酬率： -27.949 %
-27.949
1301最後資產 ： 818347 報酬率： -18.1653 %
-18.1653
3008最後資產 ： 1830860 報酬率： 83.086 %
83.086
3045最後資產 ： 669460 報酬率： -33.054 %
-33.054
3711最後資產 ： 735539 報酬率： -26.4461 %
-26.4461
4938最後資產 ： 447599 報酬率： -55.2401 %
-55.2401
2330最後資產 ： 970864 報酬率： -2.9136 %
-2.9136
5880最後資產 ： 865170 報酬率： -13.483 %
-13.483
測試結束
our總投資報酬率： -7.96544 %


In [ ]:
Weights_Dict = {}
for key in DIC_ROR:
  #print(key, '->', DIC_ROR[key])
  #篩選 validatiion 報酬率較高(n %)
  if DIC_ROR[key] > -3:
    Weights_Dict[key] = DIC_ROR[key]*DIC_ROR[key]
WeightSum = sum(Weights_Dict.values())
NewWeightSum = 0
for key in Weights_Dict:
  Weights_Dict[key] /= WeightSum
  print(key, '->', Weights_Dict[key])
  NewWeightSum += Weights_Dict[key]
print(Weights_Dict)
print(NewWeightSum)

1102 -> 0.2022090508579269
3008 -> 0.7968110992945816
2330 -> 0.000979849847491429
{'1102': 0.2022090508579269, '3008': 0.7968110992945816, '2330': 0.000979849847491429}
0.9999999999999999


In [ ]:
#決定好權重後， 開始test
#設定 股票資料
start = '2019/05/13'
# start = '2018/08/01'
# end = '2019/06/03'
end = '2020/10/29'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [ ]:
asset_at_the_start = 19 * 10**6
total_asset_present_value = 0
for stock_num in stockNum:
  if stock_num not in Weights_Dict.keys(): continue
  #先算出之後回測用的價位
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  #融資表格
  stock_time = str(Price_df.index[0]).split(" ")[0]
  # print(stock_time)
  #load in 融資融券
  url = "https://api.finmindtrade.com/api/v3/data"
  parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
  data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
  data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
  data = data.set_index('date')
  # 合併這兩個表格
  new_df = Price_df.join(data)
  new_df = new_df.drop(['stock_id'], axis=1)
  new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
  # 處理三大法人投資情況 視情況加入
  url = "https://api.finmindtrade.com/api/v3/data"
  parameter = {
    "dataset": "InstitutionalInvestorsBuySell",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  df = pd.DataFrame()
  z = 0 
  i = 0
  while i < len(data)  :
    a = data['date'][i]
    count = 0
    sell = 0
    buy = 0 
    if i+6 < len(data)-1:
        for j  in range(i, i+6):
            if data['date'][j]== a :
              count += 1
              sell += data['sell'][j]
              buy += data['buy'][j]
    else:
      for j in range(i, len(data)):
        if data['date'][j] == a :
          count += 1
          sell += data['sell'][j]
          buy += data['buy'][j]
    df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    i+=count
  new_df = new_df.join(df.set_index('date'))
  new_df = new_df.fillna(axis=0,method='ffill')
  new_df = new_df.drop(['Note'], axis=1)
  # gpy = data.groupby('date').sum()
  # gpy.rename(columns={'buy':'buy_volume', 'sell': 'sell_volume'}, inplace=True)
  # gpy['bs_ratio'] = gpy['buy_volume'] / (gpy['sell_volume']+1)
  # df = gpy
  # new_df = new_df.join(df)
  # new_df = new_df.dropna(axis=0, how='any')
  # new_df = new_df.drop(['Note'], axis=1)
  new_df.reset_index(drop=True, inplace=True)
  for columnName in new_df.columns:
    if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    # print(columnName)
    new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    days = 10
    ans = [0]*days
    for i in range(days,len(new_df)):
      Pi = new_df['Close'][i]
      Pj = new_df['Close'][i-days]
      if Pi > Pj : y = 1
      else: y = 0
      ans.append(y)
  new_df['y_label']= ans
  #更改index 將日期更改為
  new_df = new_df.reset_index()
  new_df.rename(columns={'index':'Date'}, inplace=True)
  time_list = list(new_df['Date'])
  new_df = new_df.drop(['Date'], axis=1)
  new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
  new_df = new_df.drop(['ShortSaleLimit'], axis=1)
  # new_df = new_df.drop(['level_0'], axis=1)
  Price_df = Price_df.reset_index()

  # start evaluaion 
  ###############一般回測################################
  if stock_num not in Weights_Dict.keys(): continue
  left_money = asset_at_the_start * Weights_Dict[stock_num] #10**6
  own_asset = left_money #10**6
  time_slide = 30 
  own_stock = 0 
  money_record = [left_money]*time_slide
  #origin_money = 10**6 #紀錄開場進最後出
  origin_hold_stock = 0 
  for i in range(time_slide, len(new_df)): 
    tmp = new_df[i-time_slide:i]
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model.predict(x.reshape(1,30,23)))])
    price = int(Price_df['Close'][i]*100)/100
    # if i == time_slide: #起始買
    #   origin_hold_stock += int( left_money / ( price * (1 + 0.001425)) ) # 包含手續費可以購買的股數
    #   origin_money = origin_money - origin_hold_stock*price*(1 + 0.001425)
    # elif i == len(new_df)-1: #結束賣
    #   origin_money = origin_money + origin_hold_stock*price*(1 - 0.001425 - 0.003)
    if predict == 1:
      if own_stock ==0:
        # print("買入")
        own_stock += int( left_money / ( price * (1 + 0.001425)) )
        left_money = left_money - own_stock*price*(1 + 0.001425)
        # print("目前價格： " , price, "日期: ",time_list[i])
        # print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
    elif predict == 0 :
      if own_stock > 0:
        # print("賣出")
        left_money += own_stock*price*(1 - 0.001425 - 0.003)
        own_stock = 0
        # print("目前價格： " , price, "日期: ", time_list[i])
        # print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
    money_record.append(left_money+own_stock*price)
  print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(own_asset))*100/(own_asset),"%")
  total_asset_present_value += int(money_record[-1])
print("測試結束")
print("our總投資報酬率：", (total_asset_present_value - asset_at_the_start)*100 / asset_at_the_start, "%")

1102最後資產 ： 3706297 報酬率： -3.531388763131729 %
3008最後資產 ： 12301033 報酬率： -18.74827169866875 %
2330最後資產 ： 17908 報酬率： -3.8091072624555036 %
測試結束
our總投資報酬率： -15.656642105263158 %
